# Processamento de linguagem natural (PLN) - Láb 5

In [1]:
pip install -U langchain sentence_transformers pdfplumber spacy nltk chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 23.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 KB 14.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 KB 25.0 MB/s eta 0:00:00
  Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.1/417.1 KB 26.2 MB/s eta 0:00:00
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached sqlalchemy-2.0.39-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
  Using cached asy

In [3]:
! python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [21]:
pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 24.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
Note: you may need to restart the kernel to use updated packages.


# PLN com Embedding em VectorDB

## Bibliotecas para PLN

### Chunks e embeddings

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

/home/gustavoalves/Documents/GitHub/ia_PLN/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Leitura de PDF

In [2]:
import pdfplumber

### Tratamento de texto

In [3]:
import re
import spacy
import nltk
from nltk.corpus import stopwords

(Rodar apenas uma vez)

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gustavoalves/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

---

### Bibliotecas para Banco de Dados Vetorial (Vector Database)

In [5]:
import chromadb

In [24]:
from langchain_chroma import Chroma

### Aplicando PLN - Preparação de texto

In [15]:
# Carregando PDFs

def read_pdf(pdf_path):
    pdf_reader = pdfplumber.open(pdf_path)

    page = pdf_reader.pages[0]

    text = ''

    for page in pdf_reader.pages:
        text += page.extract_text()
    
    text = text.replace('\n', " ")
    
    return text

In [28]:
# Carregando PDF

file_path = 'Classificação_de_objetos_celestes.pdf'

pdf_text = read_pdf(file_path)

In [29]:
len(pdf_text)

64064

In [30]:
pdf_text

'Classificação de Objetos Celestes February 24, 2025 1 Introdução As características espectrais são fundamentais para diferenciar distintos tipos de objetos celestes, como estrelas, galáxias e quasares. As estrelas emitem radiação eletromagnética de maneira semelhante a um corpo negro, exibindo linhas de absorção específicas. No caso das galáxias, o seu espectro é resultado da soma dos espectros de todas as estrelas e de demais materiais radiantes que as compõem. Os quasares por outro lado, apresentam linhas de emissão marcantes, frequentemente acompanhadas por um desvio expressivo para o vermelho (redshift). 2 Motivação e Contexto do DataSet OSloan Digital Sky Survey (SDSS)éumprojetodepesquisaastronômicabaseadoemimagens, conduzido com um telescópio grande angular de 2,5 metros localizado no Observatório de Apache Point, no Novo México (Estados Unidos). O objetivo do projeto é mapear um quarto do céu visível, obter observações de aproximadamente 100 milhões de objetos e o espectro de u

## PLN

In [25]:
nlp = spacy.load('pt_core_news_sm')

### Definir stopwords

In [31]:
api_stop_words = set(stopwords.words('portuguese'))

minhas_stop_words = {'a', 'e', 'i', 'o', 'u'} # Juntando minhas stopwords com as já definidas no stopwords

stop_words = api_stop_words | minhas_stop_words

In [32]:
# 

def tratamento_pln(text):

    # 1. normalização: todos os caracteres de minúsculas
    text = text.lower()

    # 2. remoção de números, pontuações e caracteres especiais
    text = re.sub(r'[^a-zA-Záéíóú\s]','', text) # na regex estão as exceções 

    # 3. tokenização com spacy 
    doc = nlp(text)
    tokens = [token.text for token in doc]

    # 4. remoção de stopwords, remoção de pontuação
    # e Lematização (clean_tokens = tokens lematizados e sem stopwords)
    clean_tokens = [token.lemma_ for token in doc if token.text not in stop_words and not token.is_punct]

    clean_text = ' '.join(clean_tokens)

    return clean_text


In [33]:
# Visualizando stop words
print("Tamanho do conjunto stop_words:", len(stop_words), "\nStop_words ordenadas: \n", sorted(list(stop_words)))

Tamanho do conjunto stop_words: 209 
Stop_words ordenadas: 
 ['a', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'essa', 'essas', 'esse', 'esses', 'esta', 'estamos', 'estar', 'estas', 'estava', 'estavam', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéramos', 'estivéssemos', 'estou', 'está', 'estávamos', 'estão', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'forem', 'formos', 'fosse', 'fossem', 'fui', 'fôramos', 'fôssemos', 'haja', 'hajam', 'hajamos', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houveram', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houvermos', 'houverá', 'houverão', 'houveríamos', 'houvesse', 'houvessem', '

In [34]:
treated_pdf_text = tratamento_pln(pdf_text)

In [36]:
print("Tamanho do texto em caracteres:",len(pdf_text))

Tamanho do texto em caracteres: 64064


In [35]:
treated_pdf_text

'classificao objeto celeste february     introduo característica espectral so fundamental diferenciar distinto tipo objeto celeste estrela galáxia quasar estrela emit radiao eletromagnético maneira semelhante corpo negro exibir linha absoro específico caso galáxia espectro resultado soma espectro todo estrela demais material radiante compir quasar outro lado apresentar linha emisso marcante frequentemente acompanhar desvio expressivo vermelho redshift   motivao contexto dataset osloan digital Sky survey sdsséumprojetodepesquisaastronmicabaseadoemimagem conduzir telescópio grande angular   metro localizar observatório Apache point novo méxico Estados Unidos objetivo projeto mapear quarto céu visível obter observaes aproximadamente   milhe objeto espectro milho obje tos levantamento incluir informaes espectral fotométrica todo objeto astronmico detectar incluir estrela galáxia quasar dado so publicar periodicamente disponibilizar publicamente conjunto dado corresponder dado data release 

#### Chunks

In [43]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=40, chunk_overlap=15)

In [44]:
chunks = text_splitter.split_text(treated_pdf_text)

In [46]:
print(chunks[:3], len(chunks))

['classificao objeto celeste february', 'february     introduo característica', 'característica espectral so fundamental'] 1769


### Modelo de embedding (all-MiniLM-L6-v2)

In [47]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [48]:
embeddings = model.encode(chunks)

In [ ]:
print(embeddings[1], len(embeddings))

### 

In [53]:
uids = [f'doc_ {i}' for i in range(len(chunks))]

### ChromaDB

In [54]:
# Criando banco de dados
client = chromadb.Client()

collection = client.create_collection(name='teste')

collection.add(documents=chunks, embeddings=embeddings, ids=uids)

In [62]:
query_embedding = model.encode(['Atributos físicos'])

In [67]:
results = collection.query(query_embeddings=query_embedding, n_results=3)

In [68]:
results

{'ids': [['doc_ 1047', 'doc_ 224', 'doc_ 1023']],
 'embeddings': None,
 'documents': [['propriedade unidade físico atributo ir',
   'anosluz via lácteo    atributo físico',
   'físico real havíamos prever grande']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None, None]],
 'distances': [[0.6899502277374268, 0.7843979597091675, 0.8140177726745605]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [69]:
for i in range(len(results['ids'][0])):
    doc_id = results['ids'][0][i]
    distance = results['distances'][0][i]
    document = results['documents'][0][i]

In [70]:
print(f"ID: {doc_id}")
print(f"Distância: {distance}")
print(f"Documento: {document}")
print("-" * 20)

ID: doc_ 1023
Distância: 0.8140177726745605
Documento: físico real havíamos prever grande
--------------------
